import data:

In [7]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
df = pd.read_csv('C:\workdir\creditcard.csv')

skálázás ás metadata:

In [8]:
from sklearn.preprocessing import RobustScaler
from sdv.metadata import SingleTableMetadata

metadata = SingleTableMetadata()
scaler = RobustScaler()

df['amount_scaled'] = scaler.fit_transform(df['Amount'].values.reshape(-1,1))
df['time_scaled'] = scaler.fit_transform(df['Time'].values.reshape(-1,1))

df.drop(['Time','Amount'], axis=1, inplace=True)
df.head()

metadata.detect_from_dataframe(data=df)
metadata.validate()

Conditions:

In [9]:
from sdv.sampling import Condition

frauds = Condition(
    num_rows=25000,
    column_values={'Class': 1}
)

non_frauds = Condition(
    num_rows=25000,
    column_values={'Class': 0}
)

synthetic data generation: CTGAN

In [10]:
from sdv.single_table import CTGANSynthesizer

ctgan_synthesizer = CTGANSynthesizer(metadata)
ctgan_synthesizer.fit(df)

ctgan_df = ctgan_synthesizer.sample_from_conditions(conditions=[frauds,non_frauds])
ctgan_df.head(10)

c:\Users\I553277\AppData\Local\Programs\Python\Python38\lib\site-packages\rdt\transformers\numerical.py:100: UserWarning: No rounding scheme detected for column 'V1'. Data will not be rounded.
  warnings.warn(
c:\Users\I553277\AppData\Local\Programs\Python\Python38\lib\site-packages\rdt\transformers\numerical.py:100: UserWarning: No rounding scheme detected for column 'V2'. Data will not be rounded.
  warnings.warn(
c:\Users\I553277\AppData\Local\Programs\Python\Python38\lib\site-packages\rdt\transformers\numerical.py:100: UserWarning: No rounding scheme detected for column 'V3'. Data will not be rounded.
  warnings.warn(
c:\Users\I553277\AppData\Local\Programs\Python\Python38\lib\site-packages\rdt\transformers\numerical.py:100: UserWarning: No rounding scheme detected for column 'V4'. Data will not be rounded.
  warnings.warn(
c:\Users\I553277\AppData\Local\Programs\Python\Python38\lib\site-packages\rdt\transformers\numerical.py:100: UserWarning: No rounding scheme detected for column

KeyboardInterrupt: 

TVAE:

In [ ]:
from sdv.single_table import TVAESynthesizer

tvae_synthesizer = TVAESynthesizer(metadata)
tvae_synthesizer.fit(df)

tvae_df = tvae_synthesizer.sample_from_conditions(conditions=[frauds,non_frauds])
tvae_df.head(10)

Evaluate the two synthetic dataframes:

In [ ]:
from sdv.evaluation.single_table import evaluate_quality

quality_report_ctgan = evaluate_quality(
    real_data=df,
    synthetic_data=ctgan_df,
    metadata=metadata)

In [ ]:
quality_report_tvae = evaluate_quality(
    real_data=df,
    synthetic_data=tvae_df,
    metadata=metadata)

In [ ]:
from sdv.evaluation.single_table import get_column_plot

fig = get_column_plot(
    real_data=df,
    synthetic_data=ctgan_df,
    column_name='Class',
    metadata=metadata
)
    
fig.show()

In [ ]:
from sdv.evaluation.single_table import get_column_plot

fig = get_column_plot(
    real_data=df,
    synthetic_data=tvae_df,
    column_name='Class',
    metadata=metadata
)
    
fig.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

print('Csalás: ', ctgan_df['Class'].value_counts()[1])
print('Nem csalás: ', ctgan_df['Class'].value_counts()[0])
sns.countplot(x=ctgan_df['Class'])
plt.show()